In [1]:
import numpy as np
import math
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
from sklearn.neighbors import NearestNeighbors
from collections import defaultdict

### Feature Descriptions
| Field Name    | Description |
| -------- | ------- |
| caid  | Unique User ID    |
| utc_timestamp | Timestamp in UTC timezone |
| id_type | Android (aaid) or iOS (idfa) |
| geo_hash | encoding geographic coordinates |
| latitude | N/S Coords |
| longitude | E/W Coords |
| horizontal_accuracy | Radius of uncertainty (meters) |
| ip_address | IP address |
| altitude | Altitude (meters?)    |
| iso_country_code | All values = US |

In [2]:
# === Load & sort ===
filepath = '00_Sample_Data/2019_Sample_Data/part-00000-85a0c7d9-db42-457d-ab9d-d104038b7a1e-c000.snappy.parquet'
data= pd.read_parquet(filepath, engine='pyarrow')

#data = data.sort_values(by=["caid", "utc_timestamp"])
data = data.sort_values(['caid', 'utc_timestamp'], kind='mergesort')

# === Remove duplicate timestamps ===
before_dedup_time = len(data)
data = data.drop_duplicates(subset=["caid", "utc_timestamp"])
after_dedup_time = len(data)
print(f"Removed {before_dedup_time - after_dedup_time} rows with duplicate timestamps per user.")

# === Remove consecutive duplicate locations, keeping first & last ===

# Convert lat/lon to NumPy once
lat = data['latitude'].to_numpy()
lon = data['longitude'].to_numpy()

# Get values for caid too
caid = data['caid'].to_numpy()

# Find run starts: lat/lon/caid != previous
diff_prev = (
    (lat != np.roll(lat, 1)) |
    (lon != np.roll(lon, 1)) |
    (caid != np.roll(caid, 1))
)
diff_prev[0] = True  # always keep first row

# Find run ends: lat/lon/caid != next
diff_next = (
    (lat != np.roll(lat, -1)) |
    (lon != np.roll(lon, -1)) |
    (caid != np.roll(caid, -1))
)
diff_next[-1] = True  # always keep last row

# Keep if start or end of run
mask = diff_prev | diff_next

# Filter in place
data = data.loc[mask]

print(f"Remaining rows: {len(data):,}")

# === Drop users with less than 10 observations ===
user_counts = data['caid'].value_counts()
valid_users = user_counts[user_counts >= 10].index
data = data[data['caid'].isin(valid_users)]
print(f"Remaining rows after dropping users with <10 rows: {len(data):,}")

# === Compute time difference in minutes within the same user ===
data["time_diff_minutes"] = data.groupby("caid")["utc_timestamp"].diff().shift(-1) / 60

# === Timestamps ===
data['datetime_utc'] = pd.to_datetime(data['utc_timestamp'], unit='s', utc=True)
data['datetime_pdt'] = data['datetime_utc'].dt.tz_convert('America/Los_Angeles')

# === Encode device type ===
data['is_iOS'] = data['id_type'].map({
    'idfa': True,
    'aaid': False
})

# === Drop uneccesary columns===
data = data.drop(columns=['id_type', 'geo_hash', 'altitude', 'iso_country_code', 'utc_timestamp', 'datetime_utc'])

# === Reorder ===
new_order = ['caid', 'datetime_pdt', 'latitude', 'longitude', 'is_iOS', 'time_diff_minutes', 'horizontal_accuracy', 'ip_address']
data = data[new_order]

# === Save ===
data.to_parquet('00_Sample_Data/2019_Pre_HDA_Data.parquet', engine='pyarrow', index=False)
# 3m 38s

Removed 0 rows with duplicate timestamps per user.
Remaining rows: 29,285,700
Remaining rows after dropping users with <10 rows: 29,194,730


### CLEANED Feature Descriptions
| Field Name    | Description |
| -------- | ------- |
| caid  | Unique User ID    |
| datetime_pdt | Timestamp in MTC region timezone |
| latitude | N/S Coords |
| longitude | E/W Coords |
| is_iOS |  iOS (True)/ Android (False) / Unknown (NaN) |
| time_diff_minutes | # of mins until the next observation for the same user |
| horizontal_accuracy | Radius of uncertainty (meters) |
| ip_address | IP address |

In [4]:
# === Core metrics ===
total_records = data.groupby('caid').size().rename('total_records')

prop_ios = data.groupby('caid')['is_iOS'].mean().rename('prop_ios').astype(float)

data['date'] = data['datetime_pdt'].dt.date
days_with_data = data.groupby('caid')['date'].nunique().rename('days_with_data')

prop_high_accuracy = (
    data.groupby('caid')['horizontal_accuracy']
    .apply(lambda x: (x <= 100).mean())
    .rename('prop_high_accuracy')
)

# === Night & bin labels ON MAIN DATA ===
data['hour'] = data['datetime_pdt'].dt.hour
data['minute'] = data['datetime_pdt'].dt.minute

data['is_night'] = ((data['hour'] >= 19) | (data['hour'] < 7))

data['night_date'] = data['datetime_pdt'].dt.date
data.loc[data['hour'] < 7, 'night_date'] -= pd.Timedelta(days=1)

# Only bin night pings
mask_night = data['is_night']
data.loc[mask_night, 'night_minute'] = data.loc[mask_night, 'hour'] * 60 + data.loc[mask_night, 'minute']
data.loc[mask_night, 'night_bin'] = data.loc[mask_night, 'night_minute'] // 30

# === Night metrics ===
temp = data.loc[mask_night, ['caid', 'night_date', 'night_bin']].copy()

# Total night pings & unique nights
night_counts = (
    temp.groupby('caid')
    .agg(
        total_night_pings=('night_date', 'count'),
        unique_nights=('night_date', 'nunique')
    )
)
night_counts['avg_night_pings_per_night'] = (
    night_counts['total_night_pings'] / night_counts['unique_nights']
)

# === Bin-level stats ===
# Bins per night per user-night
bins_per_night = (
    temp.groupby(['caid', 'night_date'])['night_bin']
    .nunique()
    .rename('bins_this_night')
).reset_index()

# Average bins per night
avg_bins_per_night = (
    bins_per_night.groupby('caid')['bins_this_night']
    .mean()
    .rename('avg_bins_per_night')
)

# === Combine all ===
user_metrics = pd.concat([
    total_records,
    prop_ios,
    days_with_data,
    prop_high_accuracy
], axis=1).reset_index()

user_metrics = (
    user_metrics
    .merge(night_counts.reset_index(), on='caid', how='left')
    .merge(avg_bins_per_night.reset_index(), on='caid', how='left')
)

# === Save ===
filepath = '00_Sample_Data/2019_Pre_HDA_Metrics.csv'
user_metrics.to_csv(filepath, index=False)
#

### User Data Quality Feature Descriptions

| Field Name                | Description |
|---------------------------|-------------|
| caid                      | Unique user ID |
| total_records             | Total number of records (rows) per user |
| prop_ios                  | Proportion of records where device type is iOS |
| days_with_data            | Number of unique days with at least one record |
| prop_high_accuracy        | Proportion of records with horizontal accuracy ≤ 100 meters |
| total_night_pings         | Total number of night-time records (7 PM–7 AM) |
| unique_nights             | Number of unique nights with at least one night-time record |
| avg_night_pings_per_night | Average number of night-time records per unique night |
| avg_bins_per_night        | Average number of unique 30-minute bins per night |

# home inference

In [7]:
# === Custom Meanshift ===
class MeanShift:
    def __init__(self, bandwidth, bin_seeding=True, min_bin_freq=2, max_iter=50):
        self.bandwidth = bandwidth
        self.bin_seeding = bin_seeding
        self.min_bin_freq = min_bin_freq
        self.max_iter = max_iter
        self.cluster_center = None
        self.used_mean = False

    @staticmethod
    def get_bin_seeds(X, bin_size, min_bin_freq=1):
        bin_sizes = defaultdict(int)
        for point in X:
            binned = np.round(point / bin_size)
            bin_sizes[tuple(binned)] += 1
        seeds = [np.array(point) * bin_size for point, freq in bin_sizes.items() if freq >= min_bin_freq]
        return np.array(seeds) if seeds else X

    @staticmethod
    def fit_single_seed(seed, X, nbrs, bandwidth, max_iter):
        stop_thresh = 1e-3 * bandwidth
        mean = seed
        for _ in range(max_iter):
            indices = nbrs.radius_neighbors([mean], bandwidth, return_distance=False)[0]
            if len(indices) == 0:
                break
            old_mean = mean
            mean = X[indices].mean(axis=0)
            if np.linalg.norm(mean - old_mean) < stop_thresh:
                break
        return tuple(mean), len(indices)

    def fit(self, X):
        if self.bin_seeding:
            seeds = self.get_bin_seeds(X, self.bandwidth, self.min_bin_freq)
        else:
            seeds = X

        nbrs = NearestNeighbors(radius=self.bandwidth).fit(X)
        results = [self.fit_single_seed(seed, X, nbrs, self.bandwidth, self.max_iter) for seed in seeds]

        clusters = {center: size for center, size in results if size > 0}

        if not clusters:
            self.cluster_center = tuple(X.mean(axis=0))
            self.used_mean = True
            return self

        self.cluster_center = max(clusters.items(), key=lambda x: x[1])[0]
        return self


# === Load Data ===
data = pd.read_parquet('00_Sample_Data/2019_Pre_HDA_Data.parquet')

# === Prepare Superpings ===
slot_size = 30 * 60  # 30 min slots
radius_m = 250  # meters
mean_lat = data['latitude'].mean()
bandwidth_deg = radius_m / 111320  # approx degrees at equator

homes = []

valid_caid = set(user_metrics['caid'])

for caid in valid_caid:
    user_df = data.loc[data['caid'] == caid]

    t = user_df['datetime_pdt'].astype(int) // 1e9
    slots = (t // slot_size).astype(int)
    user_df = user_df.assign(slot=slots)

    superpings = (
        user_df.groupby('slot')
        .agg({'latitude': 'mean', 'longitude': 'mean'})
        .dropna()
        .to_numpy()
    )

    if len(superpings) == 0:
        continue

    model = MeanShift(
        bandwidth=bandwidth_deg,
        bin_seeding=True,
        min_bin_freq=2,
        max_iter=50
    )
    model.fit(superpings)
    home_lat, home_lon = model.cluster_center

    homes.append({
        'caid': caid,
        'latitude': home_lat,
        'longitude': home_lon,
        'used_mean': model.used_mean
    })

homes_df = pd.DataFrame(homes)

KeyboardInterrupt: 

# plotting

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
bin_count = 100

# Submetric 1: Avg Night Pings per Night
axs[0].hist(user_metrics['avg_night_pings_per_night'].dropna(), bins=bin_count)
axs[0].set_title('Avg Night Pings per Night')
axs[0].set_xlabel('Avg Night Pings')
axs[0].set_ylabel('Number of Users')

# Submetric 2: Avg Bins per Night
axs[1].hist(user_metrics['avg_bins_per_night'].dropna(), bins=bin_count)
axs[1].set_title('Avg 30-min Bins per Night')
axs[1].set_xlabel('Avg Bins')
axs[1].set_ylabel('Number of Users')

# Submetric 3: Unique Nights
axs[2].hist(user_metrics['unique_nights'].dropna(), bins=bin_count)
axs[2].set_title('Unique Nights with Data')
axs[2].set_xlabel('Unique Nights')
axs[2].set_ylabel('Number of Users')

for ax in axs:
    ax.grid(True)
    ax.set_yscale('log')
plt.tight_layout()
plt.show()


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
bin_count = 50

# --- Helper to add KDE ---
def plot_hist_with_kde(ax, data, bins, title, xlabel):
    # Histogram
    counts, bins, patches = ax.hist(data, bins=bins, alpha=0.6, label='Histogram')

    # KDE
    kde = gaussian_kde(data)
    x_vals = np.linspace(min(data), max(data), 1000)
    kde_vals = kde(x_vals)
    # Scale KDE to histogram height
    kde_scaled = kde_vals * max(counts) / max(kde_vals)
    ax.plot(x_vals, kde_scaled, color='red', label='KDE')

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Number of Users')
    ax.legend()
    ax.grid(True)
    ax.set_yscale('log')

# --- Submetric 1: Log-space KDE ---
data = user_metrics['avg_night_pings_per_night'].dropna()
clip_max = 400
data_clipped = np.clip(data, 0, clip_max)  # Clip huge outliers for clean plot

# KDE in linear space, but clipped
kde = gaussian_kde(data_clipped)
x_vals = np.linspace(data_clipped.min(), data_clipped.max(), 1000)
kde_vals = kde(x_vals)

counts, bins, patches = axs[0].hist(data_clipped, bins=bin_count, alpha=0.6, label='Histogram')
kde_scaled = kde_vals * max(counts) / max(kde_vals)
axs[0].plot(x_vals, kde_scaled, color='red', label='KDE')

axs[0].set_title(f'Avg Night Pings per Night (clipped at {clip_max})')
axs[0].set_xlabel('Avg Night Pings')
axs[0].set_ylabel('Number of Users')
axs[0].legend()
axs[0].grid(True)
axs[0].set_yscale('log')

# --- Submetric 2 ---
plot_hist_with_kde(
    axs[1],
    user_metrics['avg_bins_per_night'].dropna(),
    bins=bin_count,
    title='Avg 30-min Bins per Night',
    xlabel='Avg Bins'
)
axs[1].set_xlim(0,24)
# --- Submetric 3 ---
plot_hist_with_kde(
    axs[2],
    user_metrics['unique_nights'].dropna(),
    bins=bin_count,
    title='Unique Nights with Data',
    xlabel='Unique Nights'
)

plt.tight_layout()
plt.show()


In [ ]:
# Feature to plot
feature = 'avg_night_pings_per_night'

fig, ax = plt.subplots()
ax.hist(user_metrics[feature], bins=100)
ax.set_title('Distribution of Average Night Pings per Night')
ax.set_xlabel('Average Night Pings per Night')
ax.set_ylabel('Count')
ax.grid(axis='y', zorder=0)

for bar in ax.patches:
    bar.set_zorder(2)

ax.set_xlim(left=0, right=1000)
ax.set_yscale('log')
plt.show()

In [ ]:
user_metrics['avg_night_pings_per_night'].describe()

In [ ]:
# Features to plot
features = [
    'total_night_pings',
    'unique_nights',
    'avg_night_pings_per_night',
    'avg_bins_per_night',
]

# Plot each feature
for feature in features:
    fig, ax = plt.subplots()
    ax.hist(user_metrics[feature], bins=30)
    ax.set_title(f'Distribution of {feature}')
    ax.set_xlabel(feature)
    ax.set_ylabel('Count')
    ax.grid(axis='y', zorder=0)
    for bar in ax.patches:
        bar.set_zorder(2)
    plt.show()